In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
import os

In [2]:
# Setup
# ----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

z_dim = 100
batch_size = 128
epochs = 20
lr = 5e-4
n_critic = 1
clip_value = 0.01
img_size = 28
channels = 1
img_shape = (channels, img_size, img_size)

os.makedirs("wgan_dcgan", exist_ok=True)

In [3]:
# Data Loader
# ----------------------------
transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Rescale to [-1, 1]
])

dataloader = DataLoader(
    datasets.MNIST('.', train=True, download=True, transform=transform),
    batch_size=batch_size,
    shuffle=True
)

100%|██████████| 9.91M/9.91M [00:01<00:00, 5.06MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 133kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.26MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.98MB/s]


In [4]:
# Generator
# ----------------------------
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.init_size = img_size // 4  # 7x7
        self.fc = nn.Linear(z_dim, 128 * self.init_size * self.init_size)
        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),  # 7→14
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 1, 4, stride=2, padding=1),    # 14→28
            nn.Tanh()
        )

    def forward(self, z):
        x = self.fc(z)
        x = x.view(z.size(0), 128, self.init_size, self.init_size)
        return self.conv_blocks(x)

In [5]:
# Critic
# ----------------------------
class Critic(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 64, 4, stride=2, padding=1),  # 28→14
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),  # 14→7
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Flatten(),
            nn.Linear(128 * 7 * 7, 1)  # Output is a raw score
        )

    def forward(self, x):
        return self.model(x)

In [6]:
# Initialize Models and Optimizers
# ----------------------------
generator = Generator().to(device)
critic = Critic().to(device)

optimizer_G = optim.RMSprop(generator.parameters(), lr=lr)
optimizer_C = optim.RMSprop(critic.parameters(), lr=lr)

In [7]:
# Training Loop
# ----------------------------
for epoch in range(1, epochs + 1):
    for i, (real_imgs, _) in enumerate(dataloader):

        real_imgs = real_imgs.to(device)
        b_size = real_imgs.size(0)

        # === Train Critic === #
        for _ in range(n_critic):
            z = torch.randn(b_size, z_dim, device=device)
            fake_imgs = generator(z).detach()

            loss_C = -torch.mean(critic(real_imgs)) + torch.mean(critic(fake_imgs))

            optimizer_C.zero_grad()
            loss_C.backward()
            optimizer_C.step()

            # Weight clipping for Lipschitz constraint
            for p in critic.parameters():
                p.data.clamp_(-clip_value, clip_value)

        # === Train Generator === #
        z = torch.randn(b_size, z_dim, device=device)
        gen_imgs = generator(z)
        loss_G = -torch.mean(critic(gen_imgs))

        optimizer_G.zero_grad()
        loss_G.backward()
        optimizer_G.step()

        if i % 100 == 0:
            print(f"[Epoch {epoch}/{epochs}] [Batch {i}/{len(dataloader)}] "
                  f"[Critic: {loss_C.item():.4f}] [Gen: {loss_G.item():.4f}]")

    # Save samples every epoch
    generator.eval()
    with torch.no_grad():
        z = torch.randn(64, z_dim, device=device)
        samples = generator(z)
        samples = samples * 0.5 + 0.5  # Denormalize
        save_image(samples, f"wgan_dcgan/epoch_{epoch}.png", nrow=8)
    generator.train()

[Epoch 1/20] [Batch 0/469] [Critic: 0.0799] [Gen: 0.0181]
[Epoch 1/20] [Batch 100/469] [Critic: -0.0095] [Gen: -0.0097]
[Epoch 1/20] [Batch 200/469] [Critic: -0.0137] [Gen: 0.0484]
[Epoch 1/20] [Batch 300/469] [Critic: -0.0176] [Gen: -0.0603]
[Epoch 1/20] [Batch 400/469] [Critic: -0.0159] [Gen: 0.0166]
[Epoch 2/20] [Batch 0/469] [Critic: -0.0392] [Gen: -0.0596]
[Epoch 2/20] [Batch 100/469] [Critic: -0.0283] [Gen: 0.0480]
[Epoch 2/20] [Batch 200/469] [Critic: -0.0305] [Gen: -0.0787]
[Epoch 2/20] [Batch 300/469] [Critic: -0.0073] [Gen: 0.0591]
[Epoch 2/20] [Batch 400/469] [Critic: -0.0311] [Gen: -0.0617]
[Epoch 3/20] [Batch 0/469] [Critic: -0.0167] [Gen: -0.0336]
[Epoch 3/20] [Batch 100/469] [Critic: -0.0211] [Gen: -0.0414]
[Epoch 3/20] [Batch 200/469] [Critic: -0.0207] [Gen: 0.0936]
[Epoch 3/20] [Batch 300/469] [Critic: -0.0264] [Gen: 0.0730]
[Epoch 3/20] [Batch 400/469] [Critic: -0.0251] [Gen: 0.0906]
[Epoch 4/20] [Batch 0/469] [Critic: -0.0189] [Gen: 0.0296]
[Epoch 4/20] [Batch 100/46